## CNN-LSTM IoMT IDS
### CSCI 6505 Group Project
### Author: Hongwei Zhang & Koil Jat Chong
### Enhancing Intrusion Detection in Healthcare IoMT Devices Using the CNN-LSTM Model

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import warnings
warnings.simplefilter('ignore')
import sys
print("Python version:", sys.version)
print("Version info:", sys.version_info)

Python version: 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]
Version info: sys.version_info(major=3, minor=10, micro=11, releaselevel='final', serial=0)


In [5]:
# Load the dataset
data = pd.read_csv('./dataset/processed_data.csv')
data.head()

,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,psh_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,Target
0,0.012399,0.482353,0.301176,0.000009,0.000009,0.0,0.0,0.0,0.0,0.5,...,0.465433,0.041508,9.999574e-01,0.925926,0.217273,0.466105,0.218837,1.0,1.000000,0
1,0.005670,0.417647,0.217647,0.000003,0.000003,0.0,0.0,0.1,0.0,0.5,...,0.223835,0.073505,3.288045e-10,0.333333,0.181740,0.223663,0.066195,0.9,0.153941,4
2,0.269644,1.000000,0.250980,0.000021,0.000021,0.0,0.0,0.0,0.0,0.0,...,0.187570,0.239402,3.395595e-11,0.333333,0.249725,0.187426,0.068553,0.9,0.153941,4
3,0.806794,1.000000,0.250980,0.000192,0.000192,0.0,0.0,0.0,0.0,0.0,...,0.643992,0.305027,9.996046e-01,0.925926,0.438889,0.644289,0.415955,1.0,1.000000,5
4,0.000011,0.352941,0.184314,0.000010,0.000010,0.0,0.0,1.0,0.0,0.0,...,0.148112,0.010870,9.996509e-01,0.925926,0.085875,0.148318,0.022161,1.0,1.000000,4


In [23]:
# Extract features and labels
X = data.drop(columns=['Target']).values
y = data['Target'].values
print(f'X: {X.shape}, y: {y.shape}')

X: (96282, 45), y: (96282,)


In [12]:
# Split the data into train, validation, and test sets
# 70% training, 15% validation, 15% testing
X_train, X_rest, y_train, y_rest = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_rest, y_rest, test_size=0.5, random_state=42)
print('Train set:', X_train.shape, y_train.shape)
print('Validation set:', X_val.shape, y_val.shape)
print('Test set:', X_test.shape, y_test.shape)

Train set: (67397, 45) (67397,)
Validation set: (14442, 45) (14442,)
Test set: (14443, 45) (14443,)


In [20]:
# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.int64)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.int64)

# Create data loaders
batch_size = 32
train_loader = DataLoader(TensorDataset(X_train, y_train), 
                          batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=batch_size)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=batch_size)

In [38]:
# Define the model
class CNNLSTM(nn.Module):
    def __init__(self):
        super(CNNLSTM, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(352, 64)
        #self.fc1 = nn.Linear(32 * X_train.shape[1] // 4, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 6)
        
    def forward(self, x):
        x = x.unsqueeze(1)
        x = F.relu(self.conv1(x))
        x = F.max_pool1d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool1d(x, 2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
print(CNNLSTM().to('cpu'))

CNNLSTM(
  (conv1): Conv1d(1, 16, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(16, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (fc1): Linear(in_features=352, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=6, bias=True)
)


In [39]:
# Initialize the model, loss function, and optimizer
model = CNNLSTM()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [40]:
# Training function
def train(model, dataloader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        
    return running_loss / len(dataloader.dataset)

# Evaluation function
def evaluate(model, dataloader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            
    return running_loss / len(dataloader.dataset), correct / len(dataloader.dataset)

In [41]:
# Train and evaluate the model
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer)
    val_loss, val_acc = evaluate(model, val_loader, criterion)
    print(f"Epoch {epoch+1}/{num_epochs}: <Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}, Val Acc: {val_acc:.6f}>")

Epoch 1/10: <Train Loss: 0.529603, Val Loss: 0.399585, Val Acc: 0.794142>
Epoch 2/10: <Train Loss: 0.403844, Val Loss: 0.382815, Val Acc: 0.802590>
Epoch 3/10: <Train Loss: 0.389734, Val Loss: 0.374422, Val Acc: 0.809168>
Epoch 4/10: <Train Loss: 0.381261, Val Loss: 0.369282, Val Acc: 0.810760>
Epoch 5/10: <Train Loss: 0.373945, Val Loss: 0.378682, Val Acc: 0.795665>
Epoch 6/10: <Train Loss: 0.368756, Val Loss: 0.360664, Val Acc: 0.818238>
Epoch 7/10: <Train Loss: 0.364300, Val Loss: 0.357936, Val Acc: 0.818792>
Epoch 8/10: <Train Loss: 0.360251, Val Loss: 0.356715, Val Acc: 0.823570>
Epoch 9/10: <Train Loss: 0.357135, Val Loss: 0.349318, Val Acc: 0.826409>
Epoch 10/10: <Train Loss: 0.353905, Val Loss: 0.348352, Val Acc: 0.824955>


In [43]:
# Evaluate on the test set
test_loss, test_acc = evaluate(model, test_loader, criterion)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

Test Loss: 0.3562, Test Accuracy: 0.8181
